#Sistema Multi Agente

1. Entrar no site https://groq.com/ , criar uma conta e gerar uma chave de API
2. Entrar no site https://serper.dev/signup , criar uma conta e gerar uma chave de API
3. Criar um arquivo .env com as chaves.

In [3]:
%pip install -U -q langchain_groq==0.1.9

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-cohere 0.3.1 requires langchain-core<0.4,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.
langchain 0.3.4 requires langchain-core<0.4.0,>=0.3.12, but you have langchain-core 0.2.41 which is incompatible.
langchain-openai 0.2.3 requires langchain-core<0.4.0,>=0.3.12, but you have langchain-core 0.2.41 which is incompatible.
langchain-experimental 0.3.2 requires langchain-core<0.4.0,>=0.3.6, but you have langchain-core 0.2.41 which is incompatible.
langchain-text-splitters 0.3.0 requires langchain-core<0.4.0,>=0.3.0, but you have langchain-core 0.2.41 which is incompatible.
langchain-community 0.3.3 requires langchain-core<0.4.0,>=0.3.12, but you have langchain-core 0.2.41 which is incompatible.
instructor 1.6.3 requires tenacity<10.0.0,>=9.0.0, but you have tenacity 8.5.0 which is incomp

In [4]:
%pip install -U -q python-dotenv
%pip install -U -q crewai
%pip install -U -q crewai_tools

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.9 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.3.12 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
# Sistema Multi Agente - Startup de Moda Sustentável

# Importar bibliotecas e carregar variáveis de ambiente
from dotenv import load_dotenv
import os

# Altere o caminho para o local onde o seu arquivo .env está
load_dotenv('/things.env')

from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq
from crewai_tools import SerperDevTool

# Definir o Agente Pesquisador de Mercado
market_researcher = Agent(
    role="Pesquisador de Mercado Sênior",
    goal="Realize uma análise completa do mercado de moda sustentável em São Paulo. \
           Forneça insights sobre concorrentes, tamanho de mercado, tendências de consumo \
           e potencial de crescimento.",
    backstory="Você tem vasta experiência em análise de mercado e já trabalhou com várias \
               startups de moda e sustentabilidade.",
    allow_delegation=False,
    verbose=True,
    llm="groq/llama3-8b-8192"
)

# Definir o Agente Especialista em Sustentabilidade
sustainability_expert = Agent(
    role="Especialista em Sustentabilidade",
    goal="Avalie a viabilidade sustentável do modelo de negócios proposto. \
           Verifique se o negócio está alinhado com as melhores práticas ambientais e \
           forneça recomendações para melhorar o impacto ambiental do serviço.",
    backstory="Você é especialista em sustentabilidade e trabalhou com várias marcas \
               para melhorar seus processos de produção e impacto ambiental.",
    allow_delegation=False,
    verbose=True,
    llm="groq/llama3-8b-8192"
)

# Definir o Agente Empreendedor
entrepreneur = Agent(
    role="Empreendedor",
    goal="Crie um plano de negócios e um plano de marketing para o serviço de assinatura \
           de roupas sustentáveis com base nas informações dos outros agentes.",
    backstory="Você fundou startups de sucesso e tem experiência na criação de modelos \
               de negócios inovadores.",
    allow_delegation=False,
    verbose=True,
    llm="groq/llama3-8b-8192"
)

# Ferramenta de busca para pesquisa de mercado
tool = SerperDevTool()

# Definir tarefas para o Pesquisador de Mercado
task_market_researcher = Task(
    description="Realize uma análise SWOT (Forças, Fraquezas, Oportunidades, Ameaças) \
                 do mercado de moda sustentável em São Paulo. Forneça dados sobre concorrência, \
                 tamanho de mercado e tendências de consumo.",
    expected_output=(
        "Relatório detalhado contendo análise SWOT, dados de concorrentes, tamanho de mercado \
        e tendências de consumo na moda sustentável."
    ),
    tools=[tool],
    agent=market_researcher,
)

# Definir tarefas para o Especialista em Sustentabilidade
task_sustainability_expert = Task(
    description="Avalie a viabilidade do negócio em termos de sustentabilidade. Verifique se \
                 o modelo proposto está alinhado com práticas sustentáveis e forneça recomendações \
                 para melhorar o impacto ambiental.",
    expected_output=(
        "Relatório de sustentabilidade com sugestões de melhorias e alinhamento do modelo de negócio \
        às práticas ambientais mais avançadas."
    ),
    tools=[tool],
    agent=sustainability_expert,
)

# Definir tarefas para o Empreendedor
task_entrepreneur = Task(
    description="Com base nas informações dos outros agentes, crie um plano de negócios e um plano \
                 de marketing para o serviço de assinatura de roupas sustentáveis.",
    expected_output=(
        "Plano de negócios detalhado e plano de marketing para o serviço de assinatura de roupas \
        sustentáveis em São Paulo."
    ),
    tools=[tool],
    agent=entrepreneur,
    output_file='startup_analysis.md'
)

# Criar o grupo de agentes
crew = Crew(
    agents=[market_researcher, sustainability_expert, entrepreneur],
    tasks=[task_market_researcher, task_sustainability_expert, task_entrepreneur],
    verbose=True,
    max_rpm=25
)

# Iniciar o processo com a ideia de negócio
result = crew.kickoff(
    inputs={"ideia": "Lançar um serviço de assinatura de roupas sustentáveis em São Paulo."}
)

# Verifique o tipo e os atributos do resultado
print("Tipo de resultado:", type(result))
print("Atributos do resultado:", dir(result))  # Exibir atributos do objeto result

# Verifique se result não é vazio
try:
    if result:
        # Acessar o conteúdo do resultado e verificar qual atributo contém os dados
        if hasattr(result, 'output'):
            # Se o atributo 'output' existir, usá-lo para escrever no arquivo
            with open('startup_analysis.md', 'w') as file:
                file.write(str(result.output))  # Acessa o conteúdo e escreve no arquivo
        else:
            # Caso 'output' não exista, exibir o conteúdo de __dict__
            with open('startup_analysis.md', 'w') as file:
                file.write(str(result.__dict__))  # Escreve o conteúdo dos atributos no arquivo
    else:
        print("Erro: Nenhum resultado gerado.")
except AttributeError as e:
    print(f"Erro ao acessar o resultado: {e}")


2024-10-22 13:18:02,169 - 140704740743168 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Pesquisador de Mercado Sênior
## Task: Realize uma análise SWOT (Forças, Fraquezas, Oportunidades, Ameaças)                  do mercado de moda sustentável em São Paulo. Forneça dados sobre concorrência,                  tamanho de mercado e tendências de consumo.


# Agent: Pesquisador de Mercado Sênior
## Thought: Thought: To gather information about the sustainable fashion market in São Paulo, I need to search for relevant data.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"sustainable fashion market S\\u00e3o Paulo\"}"
## Tool Output: 

Search results: Title: Brazilian Fast Fashion Market - Rural Handmade
Link: https://ruralhandmade.com/blog/brazilian-fast-fashion-market
Snippet: Established in 2017, the São Paulo-based occasion is available to general society, and elements are shown from around 50 eco-style brands, as ...
---
Title: Four Brazilian brands to showcase in Milan in sustainable fashion ...
Link: https://texbrasil.com.br/en/four-brazi

In [39]:
#Importar bibliotecas necessárias
import os

from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq
from crewai_tools import SerperDevTool
from litellm import completion

In [18]:
#Função para chamar o modelo

response = completion(
    model="groq/llama3-8b-8192",
    messages=[
        {"role": "user", "content": "hello"}
        ],
)
print(response)

ModelResponse(id='chatcmpl-61e49ed9-d48d-4494-b876-0c61e9eee93e', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", role='assistant', tool_calls=None, function_call=None))], created=1729612992, model='groq/llama3-8b-8192', object='chat.completion', system_fingerprint='fp_179b0f92c9', usage=Usage(completion_tokens=26, prompt_tokens=11, total_tokens=37, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.012135638, prompt_time=0.001259472, completion_time=0.021666667, total_time=0.022926139), service_tier=None, x_groq={'id': 'req_01jatffm8bewgtnxxqjshpfebq'})


In [41]:
#Definição do Agente "Pesquisador de Mercado"
market_researcher = Agent(
    role="Pesquisador de Mercado Sênior", #função do agente
    goal = "Garanta que o negócio {ideia} seja respaldado por pesquisas e \
    dados sólidos. Realize uma pesquisa abrangente e realista para a ideia de negócio \
    {ideia}. Forneça insights da sua pesquisa para o empreendedor.", #Objetivo da pesquisa
    backstory="Você é um especialista de mercado habilitado para pesquisas de mercado e \
    muito habilidoso em validar ideias de negócios. Você trabalhou com várias empresas \
    estabelecidas.", #histórico do agente
    allow_delegation=False, #permissão para delegar tarefas
    verbose=True, #exibir detalhes do agente
    llm="groq/llama3-8b-8192" #Modelo de linguagem usado pelo agente
)


In [42]:
#Definição do agente "Empreendedor"

enterpreneur_agent = Agent(
    role="Empreendedor experiente", #função do agente
    goal = "Criar um plano de marketing e um plano de negócios para {ideia}", #Objetivo da pesquisa
    backstory="Você construiu empresas de sucesso. Você tem habilidade de \
    criar novas ideias de negócios e planos de marketing.", #histórico do agente
    allow_delegation=False, #permissão para delegar tarefas
    verbose=True, #exibir detalhes do agente
    llm="groq/llama3-8b-8192" #Modelo de linguagem usado pelo agente
)


In [43]:
#Importação da ferramenta para buscas na web
tool = SerperDevTool()

In [44]:
#Definir as tarefas para o pesquisador de mercado
task_market_researcher = Task(
    description= " Analise os pontos fortes, fracos, oportunidades e ameaças da ideia de \
    negócio {ideia}"\
    "Estimar o tamanho do mercado  e o potencial de crescimento para essa ideia. \
    Avalie a viabilidade do modelo de negócios. \
    Avalie a existência de outras empresas com a mesma ideia no mercado. \
    Forneça insights para a criação do plano de negócios.", #tarefa a ser realizada
    expected_output=(
        "Um relatório detalhado de pesquisa de mercado para a ideia mencionada {ideia}. \
        Inclua as referências a dados externos para análise de mercado."
    ), #saída esperada
    tools=[tool], #ferramenta a ser utilizada
    agent=market_researcher,
)

In [45]:
#Definir as tarefas para o empreendedor
task_enterpreneur = Task(
    description= "Crie o plano de marketing e o plano de negócios para a {ideia} \
    Garanta que não haja discrepância entre os planos gerados. \
    Verifique se todos os conceitos importantes de planos de negócio e de \
    marketing foram cobertos. ", #tarefa a ser realizada
    expected_output=(
        "A saída deve conter um plano de negócios final para a {ideia}\
        E um plano de marketing final para a {ideia}."
    ), #saída esperada
    tools=[tool], #ferramenta a ser utilizada
    agent=enterpreneur_agent,
    output_file='automacao_ecommerce_relatorio.md'
)

In [46]:
#Criação do grupo de agentes

crew = Crew(
    agents= [market_researcher, enterpreneur_agent],
    tasks=[task_market_researcher, task_enterpreneur],
    verbose = True,
    max_rpm = 25
)

2024-10-19 15:57:00,831 - 17288 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [47]:
result = crew.kickoff(
    inputs={"ideia": "Criar um e-commerce em Jaraguá do Sul, Brasil. "}
)

# Agent: Pesquisador de Mercado Sênior
## Task:  Analise os pontos fortes, fracos, oportunidades e ameaças da ideia de     negócio Criar uma empresa no ramo textil em Jaraguá do Sul, Brasil. Estimar o tamanho do mercado  e o potencial de crescimento para essa ideia.     Avalie a viabilidade do modelo de negócios.     Avalie a existência de outras empresas com a mesma ideia no mercado.     Forneça insights para a criação do plano de negócios.


# Agent: Pesquisador de Mercado Sênior
## Thought: Thought: I will need to gather information about the textile industry in Jaraguá do Sul, Brazil, including the market size, growth potential, and competition. I will also need to analyze the strengths, weaknesses, opportunities, and threats (SWOT) of the idea to determine its viability.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"textile industry in Jaragu\\u00e1 do Sul, Brazil\"}"
## Tool Output: 

Search results: Title: Textile Mills Companies in Jaragua Do Sul, San